In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121  # For CUDA 12.1

In [ ]:
!pip install -r requirements.txt

In [ ]:
!git clone https://github.com/sobiodum/ai6-gcp-bot.git

In [ ]:
import os
import sys
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import torch as th
import numpy as np
import random
from stable_baselines3 import PPO, A2C, SAC, TD3
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat_simple import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback, BaseCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Custom learning rate schedule implementation
def custom_lr_schedule(progress_remaining: float, initial_value: float = 3e-4, final_value: float = 1e-4) -> float:
    """Linear learning rate schedule."""
    return final_value + (initial_value - final_value) * progress_remaining

# Optimize for GPU usage
device = th.device("cuda" if th.cuda.is_available() else "cpu")
if device.type == "cuda":
    th.backends.cudnn.benchmark = True  # Enable cuDNN auto-tuner
    th.set_float32_matmul_precision('high')  # Use TF32 on Ampere GPUs

# Optimized number of environments for your CPU (Ryzen 9 5950X)
N_ENVS = 14  # Slightly less than physical cores to avoid overhead
EVAL_FREQUENCY = 500_000
EVAL_FREQ_ADJUSTED = int(EVAL_FREQUENCY / N_ENVS)

# Your existing paths setup
hourly_dir = "./train2/"
source_dfs = [os.path.join(hourly_dir, f) for f in os.listdir(hourly_dir) 
              if f.endswith('.parquet') and not f.startswith('.') and 'validate' not in f]
eval_path = './train/EUR_GBP_validate.parquet'
sequence = 5
saving_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/unbiased/not_norm/train2/results/'
os.makedirs(saving_path, exist_ok=True)

# Enhanced seed setting for better reproducibility
def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    th.manual_seed(seed)
    if th.cuda.is_available():
        th.cuda.manual_seed(seed)
        th.cuda.manual_seed_all(seed)
    th.backends.cudnn.deterministic = True
    th.backends.cudnn.benchmark = False

set_all_seeds(42)

class ForexTensorboardCallback(BaseCallback):
    """Custom callback for logging Forex trading metrics to tensorboard."""
    
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.episode_returns = []  # Track episode returns for averaging
        
    def _on_step(self) -> bool:
        """Called after each step in the environment."""
        # infos is a list of dictionaries, one from each parallel environment
        for info in self.locals['infos']:
            if info is None:  # Skip if no info (can happen at episode boundaries)
                continue
                
            # Log account metrics
            self.logger.record("metrics/balance", info['balance'])
            # self.logger.record("metrics/total_return_pct", info['total_return_pct'])
            # self.logger.record("metrics/net_profit", info['net_profit'])
            
            # Log trade metrics
            self.logger.record("metrics/total_pnl", info['total_pnl'])
            # self.logger.record("metrics/total_trades", info['total_trades'])
            # self.logger.record("metrics/win_rate", info['win_rate'])
            
            # Log cost metrics
            self.logger.record("metrics/transaction_costs", info['transaction_costs'])
            # self.logger.record("metrics/transaction_costs_pct", info['transaction_costs_pct'])
            
            # Log position metrics
            self.logger.record("metrics/position_size_pct", info['position_size_pct'])
            
        return True
    
    def _on_rollout_end(self) -> None:
        """Called at the end of a rollout."""
        # Episode metrics are handled automatically by stable-baselines3
        pass

class DetailedEvalCallback(EvalCallback):
    def _on_step(self) -> bool:
        """
        Performs evaluation with detailed metric logging throughout the evaluation episodes.
        """
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Store episode rewards for calculating mean
            episode_rewards = []
            episode_lengths = []
            
            # For each evaluation episode
            for _ in range(self.n_eval_episodes):
                episode_reward = 0
                episode_length = 0
                done = False
                # VecEnv reset returns just the obs
                obs = self.eval_env.reset()
                
                # Run episode until done
                while not done:
                    # Get deterministic action
                    action, _ = self.model.predict(obs, deterministic=True)
                    # VecEnv step returns (obs, reward, done, info)
                    obs, reward, done, info = self.eval_env.step(action)
                    episode_reward += reward[0]  # reward is a numpy array
                    episode_length += 1
                    
                    # Log metrics at each step
                    if info[0] is not None:  # info is a list of dicts
                        info = info[0]  # Get info dict from first env
                        self.logger.record("eval/balance", info.get('balance', 0))
                        self.logger.record("eval/total_pnl", info.get('total_pnl', 0))
                        # self.logger.record("eval/total_trades", info.get('total_trades', 0))
                        # self.logger.record("eval/win_rate", info.get('win_rate', 0))
                        self.logger.record("eval/transaction_costs", info.get('transaction_costs', 0))
                        # Dump metrics at each step
                        self.logger.dump(self.n_calls)
                
                episode_rewards.append(episode_reward)
                episode_lengths.append(episode_length)

            # Calculate mean metrics across episodes
            mean_reward = np.mean(episode_rewards)
            mean_length = np.mean(episode_lengths)
            
            self.logger.record("eval/mean_reward", mean_reward)
            self.logger.record("eval/mean_episode_length", mean_length)

            # Update best model if needed
            if self.best_model_save_path is not None:
                if self.verbose >= 1:
                    print(f"Evaluating the current model: {mean_reward:.2f}")
                
                if mean_reward > self.best_mean_reward:
                    if self.verbose >= 1:
                        print(f"New best mean reward: {mean_reward:.2f} "
                              f"(previous: {self.best_mean_reward:.2f})")
                    self.best_mean_reward = mean_reward
                    self.model.save(self.best_model_save_path)

        return True

    def _get_eval_info(self):
        """Helper method to get the last info dict from eval environment."""
        try:
            # Try to get info directly from environment
            if hasattr(self.eval_env, 'get_info'):
                return self.eval_env.get_info()
            # If that's not available, try to get it from the unwrapped env
            elif hasattr(self.eval_env, 'envs'):
                return self.eval_env.envs[0].get_info()
            return None
        except Exception as e:
            print(f"Warning: Could not get eval info: {e}")
            return None


def make_train_env(rank):
    def _init():
        env = ForexTradingEnv(
            df_paths=source_dfs,
            eval_mode=False,
            sequence_length=sequence,
        )
        env = Monitor(env)
        return env
    return _init


train_env = SubprocVecEnv([make_train_env(i) for i in range(N_ENVS)])
train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True)



def make_eval_env():
    env = ForexTradingEnv(
        df_paths=source_dfs,
        eval_path=eval_path,
        eval_mode=True,
        pair='EUR_GBP',
        sequence_length=sequence,


    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env


eval_env = make_eval_env()

eval_callback = DetailedEvalCallback(
    eval_env,
    best_model_save_path=f'{saving_path}eval_best_model_new_reward/',
    log_path=saving_path,
    eval_freq=EVAL_FREQ_ADJUSTED,
    n_eval_episodes=5,
    deterministic=True,
    render=False
)


# Optimized policy configuration for RTX 4070
policy_kwargs_gpu = dict(
    net_arch=dict(
        pi=[512, 512, 256],
        vf=[512, 512, 256]
    ),
    lstm_hidden_size=512,
    n_lstm_layers=2,
    lstm_kwargs=dict(
        dropout=0.15,
        batch_first=True,  # Important for GPU optimization
    ),
    optimizer_kwargs=dict(
        eps=1e-5,
        weight_decay=1e-5,  # L2 regularization
        betas=(0.9, 0.999)  # Adam optimizer parameters
    ),
    share_features_extractor=False,
    enable_critic_norm=True
)

# Training configuration optimized for your hardware
training_kwargs = dict(
    learning_rate=custom_lr_schedule,  # Custom schedule instead of default constant
    batch_size=2048,                   # Increased for GPU efficiency
    ent_coef=0.01,                     # Slightly increased exploration
    device=device,                     # Explicit GPU usage
    target_kl=0.015                    # Trading-specific stability
)


# Create model with optimized configuration
model = RecurrentPPO(
    'MlpLstmPolicy',
    train_env,
    verbose=1,  # Increased verbosity for better monitoring
    seed=42,
    tensorboard_log=f'{saving_path}sequence_{sequence}__PPO_1h_gpu_optimized/',
    policy_kwargs=policy_kwargs_gpu,
    **training_kwargs
)

callbacks = [
    ForexTensorboardCallback(),
    eval_callback
]

model.learn(
    total_timesteps=30_000_000,  # Adjust as needed
    callback=callbacks
)

# Save the final model and normalizer
model.save(f'{saving_path}{sequence}_best_model_gpu_optimized.zip')
train_env.save(f'{saving_path}{sequence}_vec_normalize_gpu_optimized.pkl')
